# Simple ensembling structure - data는.. cat전


- ensemble two model
    1. Linear Regression(0.90268626150226106), lightgbm(0.87875613368405925) =>
        1. last 3
            1. Linear Regression: 0.879674
            2. lightgbm: 0.947700
            3. Random Forest: 0.881135
        2. last 6
            1. Linear Regression: 0.877625
            2. lightgbm: 0.959168
            3. Random Forest: 0.878719
        3. last 6 + X_train
            1. Linear Regression: 0.876767 => 1.00935
            2. lightgbm: 0.882532
            3. **Random Forest: 0.877934 => 1.00924**
            4. ElasticNet(alpha=0.01): 0.888654
            5. xgb: ??? => 1.01291
- ensemble three model
    1. Linear Regression, lightgbm, KnnRegressor
        - winner in two models
- ensemble five models
    1. Linear Regression, ElasticNet, lightgbm, Random Forest, KnnRegressor
        - ?

In [ ]:
X_test_level2 = np.c_[pred_lr, pred_lgb]

In [ ]:
#level2_date_blocks = [27,28,29,30,31,32]
# dates_train_level2 = dates_train[dates_train.isin(level2_date_blocks)]
#y_train_level2 = y_train[dates_train.isin(level2_date_blocks)]


# for prediction
level2_date_blocks = [28,29,30,31,32,33]
dates_train_level2 = dates_full_train[dates_full_train.isin(level2_date_blocks)]
y_train_level2 = y_full_train[dates_full_train.isin(level2_date_blocks)]


In [ ]:
# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])

# Now fill `X_train_level2` with metafeatures
for cur_block_num in tqdm_notebook(level2_date_blocks): 
    
     gc.collect()
#     lr = LinearRegression()
#     lr.fit(X_train[dates_train < cur_block_num].values, y_train[dates_train < cur_block_num])
#     pred_lr_level2 = lr.predict(X_train[dates_train == cur_block_num].values)
    
#     lgb = lightgbm.train(lgb_params, lightgbm.Dataset(X_train[dates_train < cur_block_num], label=y_train[dates_train < cur_block_num]), 100)
#     pred_lgb_level2 = lgb.predict(X_train[dates_train == cur_block_num].values)
    
#     X_train_level2[dates_train_level2.isin([cur_block_num]), 0] = pred_lr_level2.copy()
#     X_train_level2[dates_train_level2.isin([cur_block_num]), 1] = pred_lgb_level2.copy()
    
    lr = LinearRegression()
    lr.fit(X_full_train[dates_full_train < cur_block_num].values, y_full_train[dates_full_train < cur_block_num])
    pred_lr_level2 = lr.predict(X_full_train[dates_full_train == cur_block_num].values)
    
    lgb = lightgbm.train(lgb_params, lightgbm.Dataset(X_full_train[dates_full_train < cur_block_num], label=y_full_train[dates_full_train < cur_block_num]), 100)
    pred_lgb_level2 = lgb.predict(X_full_train[dates_full_train == cur_block_num].values)
    
    X_train_level2[dates_train_level2.isin([cur_block_num]), 0] = pred_lr_level2.copy()
    X_train_level2[dates_train_level2.isin([cur_block_num]), 1] = pred_lgb_level2.copy()

In [ ]:
plt.scatter(X_train_level2[:,0], X_train_level2[:,1])

In [ ]:
# X_train_plus_level2 = np.c_[X_full_train[dates_train.isin(level2_date_blocks)].values, X_train_level2]

X_train_plus_level2 = np.c_[X_full_train[dates_full_train.isin(level2_date_blocks)].values, X_train_level2]

In [ ]:
X_train_plus_level2.shape

In [ ]:
# meta_model = lightgbm.train(lgb_params, lightgbm.Dataset(X_train_plus_level2, label=y_train_level2), 100)

In [ ]:
meta_model = RandomForestRegressor(max_depth=5)

meta_model.fit(X_train_plus_level2, y_train_level2)

In [ ]:
meta_model = LinearRegression()
meta_model.fit(X_train_plus_level2, y_train_level2)

In [ ]:

meta_model = XGBRegressor(max_depth=5,learning_rate=0.03,n_jobs=-1,random_state=seed)
meta_model.fit(X_train_plus_level2, y_train_level2)

In [ ]:
X_test_plus = np.c_[X_test.values, X_test_level2]

In [ ]:
 # valid_preds = meta_model.predict(valid_plus)
# rmse_valid_stacking = rmse(y_valid, valid_preds)

print('Train rmse for stacking is %f' % rmse_train_stacking)
# print('Test  rmse for stacking is %f' % rmse_valid_stacking)

In [ ]:
ensemble_pred = meta_model.predict(X_test_plus)